In [3]:
source("external_scripts/RaceID3_StemID2_class.R")

Loading required package: tsne
Loading required package: pheatmap
Loading required package: MASS
Loading required package: cluster
Loading required package: mclust
Package 'mclust' version 5.4
Type 'citation("mclust")' for citing this R package in publications.
Loading required package: flexmix
Loading required package: lattice
Loading required package: fpc
Loading required package: amap
Loading required package: RColorBrewer
Loading required package: locfit
locfit 1.5-9.1 	 2013-03-22
Loading required package: vegan
Loading required package: permute
This is vegan 2.5-1
Loading required package: Rtsne
Loading required package: scran
Loading required package: BiocParallel
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘pack

In [1]:
x <- read.csv("source/E725.matrix.Seb_NewData_E725.3.quantif",sep="\t",header=TRUE, row.name=1)
prdata <- x[grep("ERCC",rownames(x),invert=TRUE),]

In [4]:
#dim(x)
dim(prdata)
filt <- prdata[rowSums(prdata) > 0,colSums(prdata) > 0]
dim(filt)
tab_raw <- table(prdata == 0)
tab_raw
100 * tab_raw[2] / (tab_raw[1] + tab_raw[2])

tab_fil <- table(filt == 0)
tab_fil
100 * tab_fil[2] / (tab_fil[1] + tab_fil[2])

[1] 49671   192

[1] 20467   192


  FALSE    TRUE 
1061038 8475794 

TRUE 
88.87431


  FALSE    TRUE 
1061038 2868626 

TRUE 
72.99927

In [5]:
require(biomaRt)
# Annotate the data before adding SC experiment

mart <- useMart(biomart = "ENSEMBL_MART_ENSEMBL", dataset = "mmusculus_gene_ensembl")
k <- getBM(
    filters = "ensembl_gene_id",
    attributes = c(
            "ensembl_gene_id",              # Gene stable ID
            "external_gene_name",           # Casual name
            "external_transcript_name",     # Transcript-specific name
            "gene_biotype",                 # Gene biotype
            "transcript_biotype",           # Trans type
            "description",                  # Gene description
            "band",                         # Karyotype band
            "refseq_mrna",
            "go_id",                        # Go Term accession (cellular domains)
            "go_linkage_type",              # Go Term evidence code
            "name_1006",                    # Go Term name
            "definition_1006",              # Go Term definition
            "namespace_1003"                # Go domain                  
    ),
    values = rownames(prdata),
    mart = mart
)

Loading required package: biomaRt


In [6]:
test <- merge(rownames(prdata), k, by.x = 1, by.y = 1)
head(test)

x,external_gene_name,external_transcript_name,gene_biotype,transcript_biotype,description,band,refseq_mrna,go_id,go_linkage_type,name_1006,definition_1006,namespace_1003
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0007186,IEA,G-protein coupled receptor signaling pathway,"A series of molecular signals that proceeds with an activated receptor promoting the exchange of GDP for GTP on the alpha-subunit of an associated heterotrimeric G-protein complex. The GTP-bound activated alpha-G-protein then dissociates from the beta- and gamma-subunits to further transmit the signal within the cell. The pathway begins with receptor-ligand interaction, or for basal GPCR signaling the pathway begins with the receptor activating its G protein in the absence of an agonist, and ends with regulation of a downstream cellular process, e.g. transcription. The pathway can start from the plasma membrane, Golgi or nuclear membrane (PMID:24568158 and PMID:16902576). [GOC:bf, GOC:mah, PMID:16902576, PMID:24568158, Wikipedia:G_protein-coupled_receptor]",biological_process
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0007165,IEA,signal transduction,"The cellular process in which a signal is conveyed to trigger a change in the activity or state of a cell. Signal transduction begins with reception of a signal (e.g. a ligand binding to a receptor or receptor activation by a stimulus such as light), or for signal transduction in the absence of ligand, signal-withdrawal or the activity of a constitutively active receptor. Signal transduction ends with regulation of a downstream cellular process, e.g. regulation of transcription or regulation of a metabolic process. Signal transduction covers signaling from receptors located on the surface of the cell and signaling via molecules located within the cell. For signaling between cells, signal transduction is restricted to events at and within the receiving cell. [GOC:go_curators, GOC:mtg_signaling_feb11]",biological_process
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0005525,IEA,GTP binding,"Interacting selectively and non-covalently with GTP, guanosine triphosphate. [GOC:ai]",molecular_function
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0003924,IEA,GTPase activity,Catalysis of the reaction: GTP + H2O = GDP + phosphate. [ISBN:0198547684],molecular_function
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0004871,IEA,signal transducer activity,Conveys a signal across a cell to trigger a change in cell function or state. A signal is a physical entity or change in state that is used to transfer information in order to trigger a response. [GOC:go_curators],molecular_function
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0031683,IEA,G-protein beta/gamma-subunit complex binding,"Interacting selectively and non-covalently with a complex of G-protein beta/gamma subunits. [GOC:nln, GOC:vw]",molecular_function


In [8]:
# Update rownames with external_gene_names
matrix_names <- rownames(prdata)
lookup_table <- unique(k[,c(1,3)]) # ensembl, external transcript

new_names <- c()
for (name in matrix_names){
    gene_name <- lookup_table[lookup_table$ensembl_gene_id == name,]$external_transcript_name
    # Not using external_gene_name due to isoforms in data
    if (length(gene_name) != 0){
        new_names <- c(new_names, gene_name)
    } else {
        new_names <- c(new_names, name)
    }
}

In [9]:
head(lookup_table)


,ensembl_gene_id,external_transcript_name
1,ENSMUSG00000000001,Gnai3-201
69,ENSMUSG00000000003,Pbsn-201
72,ENSMUSG00000000003,Pbsn-202
73,ENSMUSG00000000028,Cdc45-201
95,ENSMUSG00000000028,Cdc45-202
96,ENSMUSG00000000028,Cdc45-203


In [10]:
dim(lookup_table)
length(rownames(prdata))
length(new_names)

loki <- unique(k[,c(1,3)])
names_only <- unique(loki$ensembl_gene_id)



head(
    merge(names_only, loki, by.x = 1, by.y = 2, all.x = T)
)



#rownames(prdata) <- new_names
#sc <- SCseq(prdata)
#head(unique(k[,c(1,3)]))


[1] 130151      2

[1] 49671

[1] 130325

x,ensembl_gene_id
ENSMUSG00000000001,NA
ENSMUSG00000000003,NA
ENSMUSG00000000028,NA
ENSMUSG00000000031,NA
ENSMUSG00000000037,NA
ENSMUSG00000000049,NA


In [11]:
head(rownames(sc@expdata),100)


ERROR: Error in rownames(sc@expdata): object 'sc' not found


In [ ]:
# The data comes from the same batch, so we do not forsee any batch
# effects, therefore downsampling will not be used
sc <- filterdata(
    sc, mintotal=3000, minexpr=5, maxexpr=500, 
    downsample = F, sfn = F, hkn = F,
    dsn = 1, rseed = 17000, CGenes = NULL, FGenes = NULL
)
# Using defaults, we are left with 1000 genes
#dim(sc_defaults@fdata)


In [ ]:
# regress out the batch effect
# optional:
#vars <- data.frame(row.names=names(sc@fdata),batch=sub("(_|)\\d.+","",names(sc@fdata)))
#sc@fdata <- varRegression(sc@fdata,vars)

# correct for cell cycle, proliferation, and expression of degradation markers by PCA
# optional:

gCC <- name2id( k$external_gene_name[k$name_1006 == "cell cycle"],rownames(sc@fdata))
gCP <- name2id( k$external_gene_name[k$name_1006 == "cell proliferation"],rownames(sc@fdata))
vset <- list(gCC,gCP)
#g   <- sub("__chr.+","",rownames(sc@fdata));
#k   <- getBM(attributes = c("external_gene_name", "go_id","name_1006"),filters="external_gene_name",values=g,mart=mart)
#gCC <- name2id( k$external_gene_name[k$name_1006 == "cell cycle"],rownames(sc@fdata)) 
#gCP <- name2id( k$external_gene_name[k$name_1006 == "cell proliferation"],rownames(sc@fdata))
#vset <- list(gCC,gCP)

In [ ]:
k$external_gene_name[k$name_1006 == "cell cycle"]
dim(k)
dim(sc@fdata)
#x <- CCcorrect(sc@fdata,vset=vset,CGenes=NULL,ccor=.4,nComp=NULL,pvalue=.05,quant=.01,mode="pca")
#x$n


In [ ]:
# loadings of the first principal component that has been removed
y <- x$pca$rotation[,x$n[1]]
# genes from vset are either enriched in the head or the tail of this list
tail(y[order(y,decreasing=TRUE)],10)
# reassign the corrected expression matrix to sc@fdata
sc@fdata <- x$xcor

In [ ]:
# k-medoids clustering
sc <- clustexp(sc,clustnr=30,bootnr=50,metric="pearson",do.gap=T,sat=TRUE,SE.method="Tibs2001SEmax",SE.factor=.25,B.gap=50,cln=0,rseed=17000,FUNcluster="kmedoids",FSelect=TRUE)
# compute t-SNE map
sc <- comptsne(sc,rseed=15555,sammonmap=FALSE,initial_cmd=TRUE,fast=TRUE,perplexity=30)
# detect outliers and redefine clusters
sc <- findoutliers(sc, outminc=5,outlg=2,probthr=1e-3,thr=2**-(1:40),outdistquant=.95)
# reassign clusters based on random forest
sc <- rfcorrect(sc,rfseed=12345,final=TRUE,nbfactor=5)

In [ ]:
plotgap(sc)

plotsaturation(sc,disp=TRUE)
# plot change of the within-cluster dispersion as a function of the cluster number: only if sat == TRUE
plotsaturation(sc)
# silhouette of k-medoids clusters
plotsilhouette(sc)
# Jaccard's similarity of k-medoids clusters
plotjaccard(sc)
# barchart of outlier probabilities
plotoutlierprobs(sc)
# regression of background model
plotbackground(sc)
# dependence of outlier number on probability threshold (probthr)
plotsensitivity(sc)
# heatmap of k-medoids cluster
clustheatmap(sc,final=FALSE,hmethod="single")
# heatmap of final cluster
clustheatmap(sc,final=TRUE,hmethod="single")
# highlight k-medoids clusters in t-SNE map
plottsne(sc,final=FALSE)
# highlight final clusters in t-SNE map
plottsne(sc,final=TRUE)